In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import tensorflow
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook
import math
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("./data/train.csv")
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [3]:
train_df.head()

,qid,question_text,target
953466,bad31f9130e16f009bea,Will I receive messages if I blocked a number?,0
582334,72187d30b42bfa2f5ca0,How does a 25-year-old virgin straight male fe...,0
961971,bc77ce45390846c316d9,Where did the old military uniforms in the Swe...,0
305707,3be0a8168739796df6da,Which countryis better? India or Columbia,0
736958,90548c130498f60e7ab1,What are the factors that influence the choice...,0


In [5]:
# embdedding setup
# Source https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
embeddings_index = {}
f = open('./data/embeddings/glove.840B.300d/glove.840B.300d.txt', encoding='utf8')
for line in tqdm_notebook(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 2196016 word vectors.


In [7]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds += [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm_notebook(val_df["question_text"][:3000])])
val_y = np.array(val_df["target"][:3000])

In [8]:
# Data providers
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

# Training

In [9]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional

In [25]:
!tensorflow::OpRegistry::Global()

The filename, directory name, or volume label syntax is incorrect.


In [26]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(30, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=20,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Epoch 1/20


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[node bidirectional_1/CudnnRNN (defined at C:\Users\U386787\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\contrib\cudnn_rnn\python\ops\cudnn_rnn_ops.py:922)  = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](bidirectional_1/transpose, bidirectional_1/ExpandDims_1, bidirectional_1/ExpandDims_2, bidirectional_1/concat)]]

Caused by op 'bidirectional_1/CudnnRNN', defined at:
  File "C:\Users\U386787\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\U386787\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\U386787\AppData\Local\Continuum\anaconda3\lib\site-packages\ipyker